In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk 
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [24]:
#import business artical
import os
text = []
for root, dirs, files in os.walk("business"):
   for file in files:
       if file.endswith(".txt"):
            filename = os.path.join(root, file) 
            with open(filename, 'r', encoding="utf8", errors='ignore') as infile:
                intext = ''
                for line in infile:
                    intext = intext + ' ' + line.replace("\n", '')
                text.append(intext)
            


In [25]:
df = pd.DataFrame(text, columns=['text'])

In [26]:
texts = pd.Series(text)

In [27]:
texts[5]

' Japan narrowly escapes recession  Japan\'s economy teetered on the brink of a technical recession in the three months to September, figures show.  Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. A common technical definition of a recession is two successive quarters of negative growth.  The government was keen to play down the worrying implications of the data. "I maintain the view that Japan\'s economy remains in a minor adjustment phase in an upward climb, and we will monitor developments carefully," said economy minister Heizo Takenaka. But in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead, observers were less sanguine. "It\'s painting a picture of a recovery... much patchier than previously thought," said Paul Sh

In [28]:
df['text'][0]

' Ad sales boost Time Warner profit  Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up 

In [29]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e175932\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [30]:
# collect counts:
probs = {} # key: (w(t-1), w(t+1), value: {w(t): count(w(t))})
for doc in texts:
    lines = doc.split("\n")
    for line in lines:
        tokens = word_tokenize(line)
        for i in range(len(tokens) - 2):
            t_0 = tokens[i]
            t_1 = tokens[i+1]
            t_2 = tokens[i +2]
            key = (t_0, t_2)
            if key not in probs:
                probs[key] = {}
            # add count for middle token
            if t_1 not in probs[key]:
                probs[key][t_1] = 1
            else: 
                probs[key][t_1] += 1

In [31]:
# normalize probabilities 
for key, d in probs.items():
    # d should represent a distribution
    total = sum(d.values())
    for k, v in d.items():
        d[k] = v/total


In [32]:
dict(list(probs.items())[:10]) #checking probs

{('Ad', 'boost'): {'sales': 1.0},
 ('sales', 'Time'): {'boost': 1.0},
 ('boost', 'Warner'): {'Time': 1.0},
 ('Time', 'profit'): {'Warner': 1.0},
 ('Warner', 'Quarterly'): {'profit': 1.0},
 ('profit', 'profits'): {'Quarterly': 1.0},
 ('Quarterly', 'at'): {'profits': 1.0},
 ('profits', 'US'): {'at': 1.0},
 ('at', 'media'): {'US': 1.0},
 ('US', 'giant'): {'media': 0.1,
  'telecoms': 0.1,
  'banking': 0.2,
  'foods': 0.1,
  'retail': 0.1,
  'oil': 0.2,
  'mortgage': 0.1,
  'agrochemical': 0.1}}

In [33]:
df['text'].iloc[0].split("\n")

[' Ad sales boost Time Warner profit  Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL\'s underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up

In [34]:
def spin_document(doc):
    #split the document into lines(paragrah)
    lines = doc.split("\n")
    output = []
    for line in lines:
        if line:
            new_line = spin_line(line)
        else:
            new_line = line
        output.append(new_line)
    return "\n".join(output)

In [37]:
detokenizer = TreebankWordDetokenizer()

In [38]:
#df['text'].iloc[0].split("\n")[2] #checking one example

In [39]:
#detokenizer.detokenize(word_tokenize(df['text'].iloc[0].split("\n")[2]))

In [40]:
def sample_word(d):
    p0 = np.random.random()
    cummulative = 0
    for t, p in d.items():
        cummulative += p 
        if p0 < cummulative:
            return t
    assert(False) # should never get there

In [41]:
from nltk.tokenize import  word_tokenize
def spin_line(line):
    tokens = word_tokenize(line)
    i = 0 
    output = [tokens[0]]
    while i < (len(tokens) - 2):
        t_0 = tokens[i]
        t_1 = tokens[i + 1]
        t_2 = tokens[i + 2]
        key = (t_0, t_2)
        p_dist = probs[key]
        if len(p_dist) > 1 and np.random.random() < 0.3:
            #let's replace the middle word
            middle = sample_word(p_dist)
            output.append(t_1)
            output.append("<" + middle + ">")
            output.append(t_2)
            
            #we wont replace the 3rd token since the middle 
            # token was dependent on it 
            # instead, skp ahead 2 steps 
            i += 2
        else:
            # we wont replace this middle word 
            output.append(t_1)
            i += 1
    # append the final token  - only if there is no replacement
    if i == len(tokens) - 2:
        output.append(tokens[-1])
    return detokenizer.detokenize(output)

In [42]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]
new_doc = spin_document(doc)

In [45]:
import textwrap
print(textwrap.fill(
    new_doc, replace_whitespace = False, fix_sentence_endings = True
))

Unilever shake up as profit slips Anglo-Dutch consumer goods giant
Unilever is to merge <report> its two management boards after
reporting "unsatisfactory" earnings <rule> for 2004 . It blamed the
poor results on sluggish decision making, a rise <recovery> in
discounted retailers and a wet European summer . The company <IMF>
also cited difficult trading conditions and a lack <chain> of demand
<procedures> for goods such as its Slimfast <product> range .
Unilever, which <who> owns brands including Dove soap, said annual
pre-tax profit fell 36% to 2.9bn euros (£1.99bn). Shares fell 1 <0.5>%
to 510.75 <55.57> pence in London <place>, and dropped <dropped> by
1.2% to 50.50 <2.05> euros in Amsterdam . Under <But> the
restructuring <company> plans, Patrick Cescau, the UK-based co-
chairman, will become group chief executive . Dutch co-chairman Antony
Burgmans will take on <on> the role <weakness> of non-executive <its>
chairman . "We have recognised the need for greater clarity of
leadership